# Week 5: Neural Networks: Learning

## Cost Function and Backpropagation

We need a cost function for fitting the parameters of a neural network, given a training set.  

$\text{training set } = \{(x^{(1)}, y^{(1)}), (x^{(2)}, y^{(2)}), \dots, (x^{(m)}, y^{(m)})\}$  

Some new terms:  

$K$ = the total number of output units  
$L$ = the total number of layers in the network  
$s_l$ = the total number of units (excluding the bias unit) in layer $l$

### Binary Classification

$y \in \{0, 1\}$  
$K = 1$, and so $s_L = 1$  
$h_\Theta(x) \in \mathbb{R}$ (i.e. is a real number)

### Multiclass Classification

$y \in \mathbb{R}^K$, i.e. is a vector of dimension $1 \times K$

For example, if each sample could belong to one of three classes (say, an email could be family, friends or work), then:

$y \in \Bigg\{ \begin{bmatrix}
1 \\
0 \\
0 \\
\end{bmatrix}, \begin{bmatrix}
0 \\
1 \\
0 \\
\end{bmatrix}, \begin{bmatrix}
0 \\
0 \\
1 \\
\end{bmatrix}
\Bigg\}$

$s_L = K$, i.e. number of output units is same as number of classes  
$h_\Theta(x) \in \mathbb{R}^K$, i.e. same as y, as usual  

### Cost Function

Recall that the cost function for regularised logistic regression looks like this:

$$ J(\theta) = -\frac{1}{m} \bigg[\sum_{i=1}^{m} y^{(i)}\log(h_\theta(x^{(i)})) + (1 - y^{(i)})\log(1 - h_\theta(x^{(i)}))\bigg] + \frac{\lambda}{2m}\sum^n_{j = 1}\theta^2_j $$

The cost function for neural networks is similar, but with a few more nested summations:

$$ \begin{gather*} J(\Theta) = - \frac{1}{m} \sum_{i=1}^m \sum_{k=1}^K \left[y^{(i)}_k \log ((h_\Theta (x^{(i)}))_k) + (1 - y^{(i)}_k)\log (1 - (h_\Theta(x^{(i)}))_k)\right] + \frac{\lambda}{2m}\sum_{l=1}^{L-1} \sum_{i=1}^{s_l} \sum_{j=1}^{s_{l+1}} ( \Theta_{j,i}^{(l)})^2\end{gather*}$$

This is basically a generalisation of the cost function for logistic regression where instead of having one output unit, we have $K$ output units. The regularisation term is just summing over $\Theta^l_{ji}$ for all values of $j$, $i$ and $l$.

### Backpropagation Algorithm

The backpropagation algorithm is an algorithm for minimising the cost function in a neural network.  

We need to find values of $\Theta$ that minimises $J(\Theta)$, i.e. $\min_\Theta J(\Theta)$.

Suppose we have just one training sample, $(x, y)$ and a neural network that looks like this (excluding bias units):

![Neural Network](https://beths3test.s3.amazonaws.com/machine-learning-notes/nn_3_5_5_4.png)

First we perform forward propagation to get the network's prediction:

$a^{(1)} = x$  
$z^{(2)} = \Theta^{(1)}a^{(1)}$  
$a^{(2)} = g(z^{(2)})$ (don't forget to add the bias unit $a_0^{(2)}$)  
$z^{(3)} = \Theta^{(2)}a^{(2)}$  
$a^{(3)} = g(z^{(3)})$  
$z^{(4)} = \Theta^{(3)}a^{(3)}$  
$a^{(4)} = h_\Theta(x) = g(z^{(4)})$  

Then we perform backpropagation to adjust the value of $\Theta$ based on how or right wrong the prediction was.

For each node we compute the term $\delta_j^{l}$, which represents the "error" of node $j$ in layer $l$.

So for the network above we'd do the following:

$\delta^{(4)} = a^{(4)} - y$ (which is just the difference between the predicted result and the actual result)  
$\delta^{(3)} = (\Theta^{(3)})^T\delta^{(4)} \odot g'(z^{(3)})$  
$\delta^{(2)} = (\Theta^{(2)})^T\delta^{(3)} \odot g'(z^{(2)})$  

NB: $\odot$ here means element-wise multiplication.

Notice how the derivative from the subsequent layer is used in calculating the derivative of the layer before it (e.g. $\delta^{(4)}$ is used in the calculation of $\delta^{(3)}$. This is how an error is propagated back through the network.

Another interesting term is $g'$, which is the derivative of the activation function. In this case the activation function is the sigmoid function, and so $g'(z^{(l)}) = a^{(l)} \odot (1 - a^{(l)})$.

Now suppose we have a training set of $m$ samples, $\{(x^{(1)}, y^{(1)}), (x^{(2)}, y^{(2)}), \dots, (x^{(m)}, y^{(m)})\}$.


Set $\Delta^{(l)}_{i,j} := 0$ for all values of $l, i$ and $j$.

For training example i = 1:$m$,

- Set $a^{(1)} := x^{(i)}$.
- Perform forward propagation to compute $a^{(L)}$ for $l = 1, 2, 3, \dots, L$.
- Use the actual result $y^{(t)}$ to compute $\delta^{(L)} = a^{(L)} - y^{(i)}$.
- Compute $\delta^{(L-1)}, \delta^{(L-2)},\dots,\delta^{(2)}$ using $\delta^{(l)} = ((\Theta^{(l)})^T \delta^{(l+1)})\ \odot g'(z^{(l)})$ (also written as $\delta^{(l)} = ((\Theta^{(l)})^T \delta^{(l+1)})\ \odot a^{(l)}\ \odot (1 - a^{(l)})$).
- $\Delta^{(l)}_{i,j} := \Delta^{(l)}_{i,j} + a_j^{(l)} \delta_i^{(l+1)}$ or, vectorized, $\Delta^{(l)} := \Delta^{(l)} + \delta^{(l+1)}(a^{(l)})^T$.

Then, outside the for loop, we compute:

$D^{(l)}_{i,j} := \dfrac{1}{m}\left(\Delta^{(l)}_{i,j} + \lambda\Theta^{(l)}_{i,j}\right)$  
$D^{(l)}_{i,j} := \dfrac{1}{m}\Delta^{(l)}_{i,j}$  

Once all the values have been computed, $\frac \partial {\partial \Theta_{ij}^{(l)}} J(\Theta) = D^{(l)}_{i,j}$, i.e. the partial derivatives of the cost function with respect to its parameters.

We can then pass these derivatives to one of the optimisation algorithms.

### Backpropagation Intuition

Backpropagation actually works in a similar way to forward propagation, but in the opposite direction.

The cost function for a neural network is:

$$ \begin{gather*}J(\Theta) = - \frac{1}{m} \sum_{t=1}^m\sum_{k=1}^K \left[ y^{(t)}_k \ \log (h_\Theta (x^{(t)}))_k + (1 - y^{(t)}_k)\ \log (1 - h_\Theta(x^{(t)})_k)\right] + \frac{\lambda}{2m}\sum_{l=1}^{L-1} \sum_{i=1}^{s_l} \sum_{j=1}^{s_l+1} ( \Theta_{j,i}^{(l)})^2\end{gather*} $$

If we ignore regularisation and assume $K = 1$, then the cost for any given training example $(x^{(i)}, y^{(i)})$ is:

$$ cost(i) = y^{(i)} \ \log (h_\Theta (x^{(i)})) + (1 - y^{(i)})\ \log (1 - h_\Theta(x^{(i)})) $$

This cost plays a similar role to the squared error in linear regression - it measures how wrong the prediction was. So the sum of all of these costs measures how well the network as a whole is doing.

Backpropagation is computing $\delta^{(l)}_j$, which can be thought of as the "error" for $a^{(l)}_j$ - it's actually the derivative of $cost(i)$ above:

$$ \delta_j^{(l)} = \dfrac{\partial}{\partial z_j^{(l)}} cost(t) $$

That is, the derivative of the cost function for a given node with respect to $z_j^{(l)}$, which is the value passed to the activation function.

The derivative is the slope of a line tangent to the cost function, and the steeper it is the further we are from a minimum where the slope is 0, and so the more wrong we are.

These derivatives, then, are a measure of how much we'd like to change the neural network's weights in order to affect these intermediate values of the computation, so as to affect the final output of the neural network.

## Backpropagation in Practice

### Unrolling Parameters

The advanced optimisation functions like `fminunc` assume a) that the initial theta provided will be in the form of a vector and b) that the gradient return from the cost function will be a vector. To use these functions, then, we need to be able to convert our matrices to a vectors and back again.

For example, say we had a neural network such that $s_1 = 10$, $s_2 = 10$ and $s_3 = 1$.

Then:

$$\Theta^{(1)} \in \mathbb{R}^{10 \times 11}, \quad \Theta^{(2)} \in \mathbb{R}^{10 \times 11}, \quad \Theta^{(3)} \in \mathbb{R}^{1 \times 11} \\
D^{(1)} \in \mathbb{R}^{10 \times 11}, \quad D^{(2)} \in \mathbb{R}^{10 \times 11}, \quad D^{(3)} \in \mathbb{R}^{1 \times 11}$$  

In octave, the following code converts these matrices to vectors:

```octave
ThetaVec = [Theta1(:); Theta2(:); Theta3(:)];
DVec = [D1(:); D2(:); D3(:)];
```

And this code converts the vectors back to matrices:

```octave
Theta1 = reshape(ThetaVec(1:110), 10, 11);
Theta2 = reshape(ThetaVec(111:220), 10, 11);
Theta3 = reshape(ThetaVec(221:231), 1, 11);
D1 = reshape(DVec(1:110), 10, 11);
D2 = reshape(DVec(111:220), 10, 11);
D3 = reshape(DVec(221:231), 1, 11);
```

So when implementing a neural network:

- Unroll to get initial theta for `fminunc`.
- In cost function, extract `Theta1`, `Theta2` and `Theta3` from `ThetaVec` parameter.
- Use forward / backpropagation to get `D1`, `D2`, `D3` and `J`.
- Unroll `D1`, `D2` and `D3` to get `DVec`.

### Gradient Checking

$$\dfrac{\delta}{\delta\Theta}J(\Theta)\approx \frac{J(\Theta + \epsilon) - J(\Theta - \epsilon)}{2\epsilon}$$

Where $\epsilon$ is some very small value e.g. $10^{-4}$.

What this does is approximates the gradient for the current theta values by finding the gradient of the straight line joining the points $\Theta + \epsilon$ and $\Theta - \epsilon$.

With multiple $\Theta$ matrices, the formula is:

$$ \dfrac{\delta}{\delta\Theta_j}J(\Theta)\approx \frac{J(\Theta_1, \dots, \Theta_j + \epsilon, \dots, \Theta_n) - J(\Theta_1, \dots, \Theta_j - \epsilon, \dots, \Theta_n)}{2\epsilon} $$

An octave implementation of this might look like:

```octave
epsilon = 1e-4;
for i = 1:n,
  thetaPlus = theta;
  thetaPlus(i) += epsilon;
  thetaMinus = theta;
  thetaMinus(i) -= epsilon;
  gradApprox(i) = (J(thetaPlus) - J(thetaMinus))/(2*epsilon)
end;
```

We can use this to check that the values for the derivative are accurate - but turn it off before training the network for real because it's very computationally expensive.

### Random Initialisation

We need some initial value for $\Theta$. Previously we've set it to a vector of all 0s, but this won't work for neural networks.

If you set $\Theta_{ij}^{(l)} = 0$ for all $i, j, l$ then all the nodes in the network will update to the same value and all the derivatives will update to the same value. After each update, parameters corresponding to inputs going into each of the hidden units are identical.

This is known as the problem of symmetric weights. We can fix it by doing some symmetry breaking. This can be achieved by setting each $\Theta_{ij}^{(l)}$ to some random value in $[-\epsilon \epsilon]$. (Nothing to do with the $\epsilon$ in the section above.)

In octave:

```octave
% If the dimensions of Theta1 is 10x11, Theta2 is 10x11 and Theta3 is 1x11.
Theta1 = rand(10,11) * (2 * INIT_EPSILON) - INIT_EPSILON;
Theta2 = rand(10,11) * (2 * INIT_EPSILON) - INIT_EPSILON;
Theta3 = rand(1,11) * (2 * INIT_EPSILON) - INIT_EPSILON;
```

### Putting It Together

Pick a neural network architecture:

- Number of input units = number of features $x^{(i)}$.
- Number of output units = number of classes.
- Number of hidden units = more is generally better (although more computationally expensive); have at least as many as input units.
- Default to one hidden layer.

More on designing an architecture in coming weeks!

Training a neural network:

- Randomly initialise the theta values / weights.
- Forward propagation to get predictions.
- Implement the cost function.
- Backpropagation to get the partial derivatives.
- Check backpropagation working properly with gradient checking -> then turn off gradient checking.
- Pass cost function and derivatives to some advanced optimisation function e.g. `fminunc` to minimise the cost function with the weights in theta.

N.B. $J(\Theta)$ is not convex, so you might end up in a local minimum.